In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import time

In [ ]:
def comparison(threshold,train,test,information_type):
    if information_type =="prediction_var":
        return ((train >= threshold).sum() + (test < threshold).sum()) / (len(train)+len(test)) 
    else:
        return ((train <= threshold).sum() + (test > threshold).sum()) / (len(train)+len(test))

def read_model(path,model_number, information_type,cifar=False):
    if not cifar:
        df = pd.read_csv("{}/{}.csv".format(path,model_number),index_col=0)
        train = df["train_{}".format(information_type)]
        test = df["test_{}".format(information_type)]
    else:
        df = pd.read_csv("{}/{}_train.csv".format(path,model_number),index_col=0)
        train = df["train_{}".format(information_type)]
        train = train.head(10000)
        df = pd.read_csv("{}/{}_test.csv".format(path,model_number),index_col=0)
        test = df["test_{}".format(information_type)]
    return train, test

def evaluate_optimal_attack(path,model_number,information_type,cifar=False):
    train, test = read_model(path,model_number, information_type,cifar)
    optimal_acc = 0
    optimal_threshold = np.nan
    sort_train = np.sort(np.unique(train))
    #We're not looking at every possible thrshold but only every 10th, 
    #that speeds up computation by 10 times and doesn't seem to have
    #effect on the performance
    for threshold_ind in range(0,len(sort_train),10):
        threshold = sort_train[threshold_ind]
        acc = comparison(threshold,train,test,information_type)
        if acc>optimal_acc:
            optimal_acc = acc
            optimal_threshold = threshold
    return optimal_acc, optimal_threshold

def evaluate_k_shadow_attack(path,model_number,information_type,k,cifar=False,max_model_number=4):
    if cifar: return np.nan,np.nan
    thresholds = np.zeros(k)
    for current_k in range(k):
        _, thresholds[current_k] = evaluate_optimal_attack(path,(model_number+1+k)% max_model_number,information_type)
    threshold = np.median(thresholds)
    train, test = read_model(path,model_number, information_type,cifar=cifar)
    acc = comparison(threshold,train,test,information_type)
    return acc, threshold     

def evaluate_split_attack(path,model_number,information_type,split=0.5,cifar=False):
    train, test = read_model(path,model_number, information_type,cifar=cifar)
    all_data = np.concatenate([train, test])
    threshold = np.quantile(all_data,split)
    acc = comparison(threshold,train,test,information_type)
    return acc, threshold

## Evaluation of Lime attacks

In [ ]:
cifar=False
experiment_numbers = np.arange(5)
max_model_number = 4
path="ThresholdExperiments/{}/{}".format(experiment_name,random_state)

In [ ]:
%%time
for experiment_name in ["purchase-lime", "purchase-smoothgrad-1000", "purchase-1000"]:
    information_types = ["loss", "prediction_var", "analysis_var"]
    model_numbers = np.arange(max_model_number)
    result_accs, result_thresholds = {},{}
    attack_types = {'optimal':[],"shadow_1":[],"shadow_3":[],"split":[]}
    for information_type in information_types:
        result_accs[information_type] = copy.deepcopy(attack_types)
        result_thresholds[information_type] = copy.deepcopy(attack_types)
    print(experiment_name, end=" ")
    for attack_type in ['optimal\t',"shadow_1","shadow_3","split"]:
            print("{}\t".format(attack_type),end=" ")
    counter = 0
    start = time.time()
    for experiment_numer in experiment_numbers:
        path="ThresholdExperiments/{}/{}/".format(experiment_name,experiment_numer)
        for model_number in model_numbers:
            #print(counter, end= " ")
            counter += 1
            for information_type in ["analysis_var", "prediction_var"]: 
                #print(information_type, end=" ")
                acc, threshold = evaluate_optimal_attack(path,model_number,information_type,cifar=cifar)
                result_accs[information_type]["optimal"].append(acc)
                result_thresholds[information_type]["optimal"].append(threshold)
                acc, threshold = evaluate_k_shadow_attack(path,model_number,information_type,k=1,cifar=cifar,max_model_number=2)
                result_accs[information_type]["shadow_1"].append(acc)
                result_thresholds[information_type]["shadow_1"].append(threshold)
                acc, threshold = evaluate_k_shadow_attack(path,model_number,information_type,k=3,cifar=cifar)
                result_accs[information_type]["shadow_3"].append(acc)
                result_thresholds[information_type]["shadow_3"].append(threshold)
                acc, threshold = evaluate_split_attack(path,model_number,information_type,split=spilt,cifar=cifar)
                result_accs[information_type]["split"].append(acc)
                result_thresholds[information_type]["split"].append(threshold)
    print(time.time() - start) 
    pickle.dump(result_accs,open(experiment_name+"_accs","bw"))
    pickle.dump(result_thresholds,open(experiment_name+"_thrs","bw"))
    for information_type in ["analysis_var", "prediction_var"]: 
        print(information_type, end="\t")
        if information_type == "loss":
            print("\t",end="")
        for attack_type in ['optimal',"shadow_1","shadow_3","split"]:
            print("{:.3f}\t\t".format(np.mean(result_accs[information_type][attack_type])),end=" ")
        print("")

In [ ]:
%%time
for experiment_name in ["texas-lime", "texas-smoothgrad-1000", "texas-1000"]:
    information_types = ["loss", "prediction_var", "analysis_var"]
    model_numbers = np.arange(max_model_number)
    result_accs, result_thresholds = {},{}
    attack_types = {'optimal':[],"shadow_1":[],"shadow_3":[],"split":[]}
    for information_type in information_types:
        result_accs[information_type] = copy.deepcopy(attack_types)
        result_thresholds[information_type] = copy.deepcopy(attack_types)
    print(experiment_name, end=" ")
    for attack_type in ['optimal\t',"shadow_1","shadow_3","split"]:
            print("{}\t".format(attack_type),end=" ")
    counter = 0
    start = time.time()
    for experiment_numer in experiment_numbers:
        path="ThresholdExperiments/{}/{}/".format(experiment_name,experiment_numer)
        for model_number in model_numbers:
            #print(counter, end= " ")
            counter += 1
            for information_type in ["analysis_var", "prediction_var"]: 
                #print(information_type, end=" ")
                acc, threshold = evaluate_optimal_attack(path,model_number,information_type,cifar=cifar)
                result_accs[information_type]["optimal"].append(acc)
                result_thresholds[information_type]["optimal"].append(threshold)
                acc, threshold = evaluate_k_shadow_attack(path,model_number,information_type,k=1,cifar=cifar,max_model_number=2)
                result_accs[information_type]["shadow_1"].append(acc)
                result_thresholds[information_type]["shadow_1"].append(threshold)
                acc, threshold = evaluate_k_shadow_attack(path,model_number,information_type,k=3,cifar=cifar)
                result_accs[information_type]["shadow_3"].append(acc)
                result_thresholds[information_type]["shadow_3"].append(threshold)
                acc, threshold = evaluate_split_attack(path,model_number,information_type,split=spilt,cifar=cifar)
                result_accs[information_type]["split"].append(acc)
                result_thresholds[information_type]["split"].append(threshold)
    print(time.time() - start) 
    pickle.dump(result_accs,open(experiment_name+"_accs","bw"))
    pickle.dump(result_thresholds,open(experiment_name+"_thrs","bw"))
    for information_type in ["analysis_var", "prediction_var"]: 
        print(information_type, end="\t")
        if information_type == "loss":
            print("\t",end="")
        for attack_type in ['optimal',"shadow_1","shadow_3","split"]:
            print("{:.3f}\t\t".format(np.mean(result_accs[information_type][attack_type])),end=" ")
        print("")